In [1]:
import os
keys = []
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                keys.append(value)

# API 키 로드 및 환경변수 설정
load_api_keys('api_key.txt')

In [2]:
from typing import List, Dict, Optional, TypedDict, Any
from fastapi import FastAPI , Body ,HTTPException
import torch
import ast
import json
import uuid
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.genai import types
from fastapi.middleware.cors import CORSMiddleware

## 모델 로딩

### 재미나이 로딩

In [3]:
def keyChange():
    global client
    from google import genai
    tt = keys.pop(0)
    keys.append(tt)
    client = genai.Client(api_key=tt)

keyChange()

### 백터DB와 임베딩 모델 (백터 DB : PGVectorDB,임베딩모델:ko-sroberta-multitask)

In [4]:
model_vector = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 2. 원격 pgvector 연결 설정
def get_db_connection():
    conn = psycopg2.connect(
        host="100.95.214.66",
        database="postgres",
        user="a085009",
        password="aivle202508",
        port=5432
    )
    register_vector(conn)
    return conn



## State

In [5]:
class State(TypedDict, total = False):
    txt: str  #front
    episode : int  #front 
    subtitle : str #front
    check : dict
    title : str    #front
    userid : str    #front
    캐릭터 : dict
    세계관 : dict
    장소 : dict
    사건 : dict
    아이템 : dict
    단체 : dict

## 백터DB 삽입, 조회, 수정, 삭제

In [6]:
def get_vector(text):
    return model_vector.encode([text])[0]

def upload_vector(state:State,lis): #상태와 업데이트 목록 /가장 마지막순서 : 신규 업데이트일 경우
    """
lis = {
'캐릭터':['캐릭터1','캐릭터2',...],
'세계관':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'아이템':['아이템1',....],
'단체':['단체1','단체2']
    }
    """
    conn = get_db_connection()
    cur = conn.cursor()
    episode =state.get("episode",0)
    subtitle = state.get("subtitle")
    for key,values in lis.items():
        for value in values:
            try :
                settings_data = {value : state.get(key).get(value)}
                text_to_embed = f"{key} : {state.get(key).get(value)} /{episode}화, 부제목 : {subtitle}"
                embedding_vector = get_vector(text_to_embed)
                    # 4. 데이터 삽입 (vector 컬럼은 768차원이어야 합니다)
                insert_sql = """
                INSERT INTO setting ( userid, title, tag, keyword, episode, subtitle, settings, embedding)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                cur.execute(insert_sql, (
                    [state.get("userid","aaaaaaaa")], 
                    state.get("title","제목없음"),
                    key,
                    value,
                    [episode],
                    subtitle,
                    json.dumps(settings_data), 
                    embedding_vector # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
                ))
                conn.commit()
                print("성공적으로 저장되었습니다!")
            except Exception as e:
                print(e)
                return "500 Internal Server Error"  # DB연결실패
    cur.close()
    conn.close()
    return "Success! New Setting upload"

### 청킹

In [7]:
def chunking(state:State):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150,        # 목표 청크 크기
        chunk_overlap=50,      # 겹침 크기
        separators=["\n\n", "\n", ".", " ", ""] # 자르는 기준 우선순위
    )
    
    chunks = text_splitter.split_text(state.get('txt'))
    return chunks

### 원문 삽입

In [8]:
def upload_original(state:State): #원문 업로드
    chunks = chunking(state)
    conn = get_db_connection()
    cur = conn.cursor()
    episode =state.get("episode")
    subtitle = state.get("subtitle")
    userid = state.get("userid","작가미상"), 
    title = state.get("title","제목없음"),
    for txt in chunks:
        try :
            embedding_vector = get_vector(txt)
                # 4. 데이터 삽입 (vector 컬럼은 384차원이어야 합니다)
            insert_sql = """
            INSERT INTO original ( userid, title, episode, subtitle, txt, embedding)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            cur.execute(insert_sql, (
                userid,
                title,
                episode,
                subtitle,
                txt,
                embedding_vector # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
            ))
            conn.commit()
            print("성공적으로 저장되었습니다!")
        except Exception as e:
            print(e)
            return "500 Internal Server Error"  # DB연결실패
    cur.close()
    conn.close()

### 유저검색

In [9]:
def userQuestion(tag,user_query,state,sim,limit):  # 태그(*,캐릭터,장소,사건,세계관,등)
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    userid = state.get('userid','작가미상')
    title = state.get('title','제목없음')
    search_sql = """
    SELECT * FROM (
        SELECT 
            id,
            settings, 
            1 - (embedding <=> %s::vector) AS similarity 
        FROM setting 
        WHERE embedding IS NOT NULL 
            AND %s = ANY(userid)
            AND title = %s
            AND (%s = '*' OR tag = %s)
    ) sub
    WHERE similarity >= %s  -- 여기서 원하는 유사도 문턱값을 설정 (예: 0.3 이상)
    ORDER BY similarity DESC  
    LIMIT %s;
    """
    cur.execute(search_sql, (query_vector,userid,title,tag,tag,sim,limit))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

In [10]:
# state['userid'] = "aaaaaaaa"
# userQuestion("*","비에고의 아내 죽음",state,0.2,10)

In [11]:
def originalQ(state,user_query):
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    userid = state.get('userid','aaaaaaaa')
    title = state.get('title','제목없음')
    search_sql = """
    SELECT 
        id,
        txt, 
        1-(embedding <=> %s::vector) AS distance 
    FROM original 
    WHERE embedding IS NOT NULL 
        AND userid = %s
        AND title = %s
    ORDER BY distance DESC  
    LIMIT 2;
    """
    cur.execute(search_sql, (query_vector,userid,title))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

### 일반검색

In [12]:
def select(tag,keyword,title,userid): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor()
    search_sql= """
    SELECT 
        id,
        settings,
        episode
    FROM setting 
    WHERE %s = ANY(userid)
        AND title = %s
        AND tag = %s
        AND keyword = %s
    """
    cur.execute(search_sql, (userid,title,tag,keyword))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result




In [13]:
# a = select("캐릭터","비에고","리그오브레전드 스토리","{미상}")
# print(a[0])

### 수정

In [14]:
def edit(db_id,new_setting,episode):
    conn = get_db_connection()
    cur = conn.cursor()
    new_vector = f"{new_setting} /{episode}화"
    embedding_vector = get_vector(new_vector)
    update_sql = """
    UPDATE setting 
    SET 
        episode = %s,
        settings = %s,
        embedding = %s::vector
    WHERE id = %s;
    """
    try:
        cur.execute(update_sql, (episode,json.dumps(new_setting), embedding_vector, db_id))
        conn.commit()
        print(f"데이터가 성공적으로 수정되었습니다.")
    except Exception as e:
        conn.rollback()
        return (f"수정 실패: {e}")
    finally:
        cur.close()
        conn.close()
    return "수정 성공"

### 삭제

In [15]:
def delete(char_id):
    conn = get_db_connection()
    cur = conn.cursor()
    
    # 기본키를 조건으로 삭제
    delete_sql = "DELETE FROM setting WHERE id = %s;"
    
    try:
        cur.execute(delete_sql, (char_id,))
        conn.commit()
        print(f"ID {char_id} 삭제 완료.")
    except Exception as e:
        conn.rollback()
        print(f"삭제 실패: {e}")
    finally:
        cur.close()
        conn.close()

## 메시지 입력

In [16]:
count_change= 0
# def genai(system_prompt,msg):
#     global count_change
#     try:
#         response = client.models.generate_content(
#             model='gemini-3-flash',   #"gemini-2.5-flash",  #gemini-2.5-flash-lite
#             config=types.GenerateContentConfig(
#                 system_instruction=system_prompt, # 여기에 시스템 프롬프트를 넣습니다.
#                 temperature=0.1, # 데이터 추출이므로 낮게 설정
#             ),
#             contents=msg
#         )
#         return response.text
#     except Exception as e:
#         count_change +=1
#         if count_change >=30:
#             return "모두다씀"
#         keyChange()
#         return genai(system_prompt,msg)

def genai(system_prompt,msg): 
    global count_change
    try:
        response = client.models.generate_content(
            model="gemma-3-27b-it", #gemma-3-12b, gemma-3-1b ,gemma-3-2b, gemma-3-4b
            contents=system_prompt+msg,
        )
        return response.text
    except Exception as e:
        count_change +=1
        if count_change >=30:
            return "모두다씀"
        keyChange()
        return genai(system_prompt,msg)

In [17]:
# print(genai("ㅎㅇ","test"))
# print(os.environ['GEMINI_API_KEY'])

## 1. 카테고리 추출

In [18]:
def Categories(state: State) -> State:
    system_prompt="""### Role
너는 소설 원문에서 설정 데이터를 누락 없이 추출하여 구조화하는 전문 데이터 분석가입니다.

### Task
당신은 '본문'의 글을 정보를 추출하기 쉽도록 요약하고자 합니다. 
요약한 글에서 추출할 정보는 '캐릭터, 세계관, 장소, 사건, 아이템, 단체'에 관한 내용입니다.
해당 내용 보전을 중심으로 요약하고 추출해야 할 목록을 딕셔너리{} 텍스트 형태로 만들고자 합니다.

### Output Format 
{
'캐릭터':['캐릭터1','캐릭터2',...],
'세계관':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'아이템':['아이템1',....],
'단체':['단체1','단체2'],
}

### Constraints
- 특정 주인공 중심의 요약을 하지 마세요. 등장하는 모든 고유 명사를 분류하세요.
- 불릿 포인트 및 볼드체를  사용하지 마세요.
- **키워드가 중복되지 않도록 주의하세요.
- **추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
- **본문에 명확히 나오지 않은 정보는 빈 배열 `[]`로 처리하세요.
- 키워드에 수식어를 달지 마세요.
"""
    msg = f"본문: {state.get('txt','')} \n 위 본문을 분석하여 '캐릭터, 세계관, 장소, 사건, 아이템, 단체' 정보를 ```json 같은 마크다운 태그를 제외하고 내용만 보내줘."
    
    result = genai(system_prompt,msg)
    print("목록 추출 완료")
    print("=="*8)
    print(result)
    print("=="*8)
    try:
        st = result.index('{')
        ed = result.index('}')
        result = ast.literal_eval(result[st:ed+1])
    except:
        print("딕셔너리 변환 에러")
        state = Categories(state)
    return {**state,
            'check' : result
           }

## 설정 추출

### 캐릭터 추출

In [19]:
def charter(state: State) -> State:
    lis = state.get('check')
    lis = lis['캐릭터']
    txt = state.get('txt')
    chart = state['캐릭터']
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 캐릭터 설정집(Character Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 등장인물 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요
# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"""+f""" 
  "{ix}" """+""": [
    { "별명": "본문에 명시된 별명 (없으면 Unknown)",
      "정체성": {
        "종족": "종족 (없으면 Unknown)",
        "연령": "나이 또는 연령대 (없으면 Unknown)",
        "직업/신분": "신분 (없으면 Unknown)",
        "소속단체/가문": "소속 (없으면 Unknown)"
      },
      "외형": "외모 묘사 (없으면 Unknown)",
      "성격": ["성격 키워드1 (없으면 Unknown)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 Unknown)", "숙련도/강도": "수준 (없으면 Unknown)", "상세효과": "효과설명 (없으면 Unknown)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 Unknown)", "관계": "관계 정의( 없으면 Unknown"), "상세내용": "관계 설명(없으면 Unknown)"}
      ]
    }
  ]
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요."""
        
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            chart[ix] = result[ix]
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
        
    return {**state,
           '캐릭터' :chart
           }

### 세계관 추출

In [20]:
def rule(state: State) -> State:
    lis = state.get('check')
    lis = lis['세계관']
    txt = state.get('txt')
    rule = state['세계관']
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 세계관 설정집을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 세계관 키워드 {ix}을/를 본문 없이 작품 속 세계의 규칙을 이해할 수 있도록 300자 이내로 설명하는 문장을 만들어 JSON 포맷으로 출력하세요.

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 요약은 가능하지만 본문의 내용을 바탕으로 추측 없이 문장을 생성하세요.
2. 연관된 이야기 없이 대상에 대한 내용만으로 짧게 요약하여 문장을 생성하세요.

"""+"""   
# JSON_Output_Format
{"""+f""" "{ix}" """+""": "문장"}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 명확히 나오지 않은 정보는 "Unknown" 또는 빈 배열 `[]`로 처리하세요."""
        
        msg= f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."           
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            rule[ix] = result[ix]
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '세계관' :rule
           }

### 장소 추출

In [21]:
def place(state: State) -> State:
    lis = state.get('check')
    lis = lis['장소']
    txt = state.get('txt')
    place = state['장소']
    for ix in lis:
        system_prompt=  f"""# Role
귀하는 소설의 원문을 분석하여 장소 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"""+f""" 
  "{ix}" """+""": 
    { "별칭": "지역별칭 (없으면 Unknown)"
      "지역정체성": {
        "분위기": "지역의 분위기(없으면 Unknown)",
        "역사": "지역의 역사 또는 생성된 시기 (없으면 Unknown)",
        "위치": "지역의 위치 (없으면 Unknown)"
      },
      "단체": ["해당 공간에 속한 소속 단체/가문(없으면 Unknown)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 Unknown)"]
    }
  
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요."""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
                    

        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            place[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '장소' :place
           }

### 사건 추출

In [22]:
def event(state: State) -> State:
    lis = state.get('check')
    lis = lis['사건']
    txt = state.get('txt')
    event = state['사건']
    
    for ix in lis:
        system_prompt= f"""# Role
귀하는 소설의 원문을 분석하여 사건 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 사건 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
설명에는 해당 사건에 관해 간략한 원인과 결과를 한줄요약하여 써야한다

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": "해당 사건에 관한 짧은 설명"}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
"""
        msg=f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            event[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '사건' :event
           }

### 아이템 추출

In [23]:
def item(state: State) -> State:
    lis = state.get('check')
    lis = lis['아이템']
    txt = state.get('txt')
    item = state['아이템']
    
    for ix in lis:
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 게임의 아이템 설정(item Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 아이템 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 아이템의 성능과 역할에 대해 간략하게 적어야 합니다.
character에는 해당 사건의 메인 인물을 적어야 합니다.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": {"설명":"해당 아이템 관한 짧은 설명 (없으면 Unknown)",
"관련인물" :["관련된인물 (없으면 Unknown)"] }}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 관련인물은 사건에 등장하는 모든 인물이 아닌 사건의 주역만 적어야 합니다. 주역이 없을 경우 빈 배열 '[]'로 처리하세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            item[ix] = result[ix]

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '아이템' :item
           }

### 단체 추출

In [24]:
def group(state: State) -> State:
    lis = state.get('check')
    lis = lis['단체']
    txt = state.get('txt')
    group = state['단체']
    
    for ix in lis:
        system_prompt=f"""# Role
귀하는 소설의 원문을 분석하여 단체 설정(group Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 "Unknown"을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 단체 {ix}의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 단체의 역사와 목적 대해 간략하게 적어야 합니다.
character에는 해당 단체에 속해있는 인물을 적은 후 단체에서 역할을 괄호'()'안에 적으세요.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{"""+f'"{ix}"'+""": {"설명":"단체에 관한 짧은 설명(없으면 Unknown)",
"구성원" :["구성원(역할)(없으면 Unknown)"] }}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 "Unknown"으로 남겨두세요.
- key값이나 character 의 구성원 value값은 오타나 언어의 변환 없이 본문 그대로 넣으세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            group.update(result)

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '단체' :group
           }

## 추출한 설정 검수 & 분류

In [25]:
def comparison(state: State):
    check = state.get('check')
    title = state.get('title')
    userid = state.get('userid')
    uplist ={'캐릭터':[],'세계관':[],'장소':[],'사건':[],'아이템':[],'단체':[]}
    editlist = {'캐릭터':[],'세계관':[],'장소':[],'사건':[],'아이템':[],'단체':[]}
    errorlist = {'캐릭터':[],'세계관':[],'장소':[],'사건':[],'아이템':[],'단체':[]}
    for tag,val in check.items():
        for keyword in val:
            emb = select(tag,keyword,title,userid)
            if (len(emb)==0):
                uplist[tag].append(keyword) 
                continue
            i = state.get(tag).get(keyword)
            system_prompt= f"""#### 역할: 소설 설정 검수 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 설정이 어긋나는 경우가 존재하는지 찾아보세요.

### 분석할 설정
기존설정 : {emb[0][1]}

신규설정 : {i}

### 요청 사항
2. **상세 분석:** - **인과관계/상세화:** 한쪽이 원인이고 다른 쪽이 결과이거나, 한쪽이 다른 쪽의 상세 설명인 경우 '결합'로 판단합니다.
   - **충돌:** 예외 없이 물리적으로 불가능한 경우(예: 죽었는데 살아있음, 파란 눈인데 빨간 눈임)에만 '충돌'로 판단하세요.
    -**일치:** 설정이 추가되거나 보강되는 부분 없이 동일한 경우에는 '일치'로 판단하세요.
    
### 출력 형식
[결과: 충돌/결합/일치]
[판단사유: 판단사유]
        """
            msg = f"이거 설정이 맞을까?"     
            result = genai(system_prompt,msg)
            if "[결과: 충돌]" in result:
                errorlist[tag].append([keyword,result])
            elif "[결과: 결합]" in result:
                editlist[tag].append(emb)
            print(result)
    editlist = edit_setting(editlist,state)
    return {'충돌': errorlist, '설정 결합' : editlist, "신규 업로드": uplist }

In [26]:
def edit_setting(editlist, state:State):
    lis = []
    for tag,value in editlist.items():
        for emb in value:            
            vctid,oldset,episode = emb[0]
            keyword = next(iter(oldset))
            system_prompt= f"""#### 역할: 소설 설정 작성 전문가
            
    ### 상황
    하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 두 설정을 결합하세요.
    
    ### 분석할 설정
    기존설정 : {oldset}
    
    신규설정 : {keyword} : {state.get(tag).get(keyword)}
    
    ### 출력 형식
    설정 형식과 동일
    """
            msg = f"두 설정을 결합해줄래"     
            result = genai(system_prompt,msg)
            try:
                result = result.replace("json", "").replace("```", "").strip()
                result = json.loads(result)
            except:
                print("딕셔너리 변환 에러 : edit_setting")
                print(result)
                #state = charter(state)
            print(result)
            episode.append(state.get('episode'))
            lis.append([vctid,result,episode])
    return lis

## Agent - Rang Graph 

In [27]:
from langgraph.graph import StateGraph, END

### 카테고리 추출 

In [28]:
cat_builder = StateGraph(State)
cat_builder.add_node("Categories",Categories)

cat_builder.set_entry_point("Categories")
cat_builder.add_edge("Categories",END)

cat = cat_builder.compile()


### 상세설정 추출 

In [29]:
builder = StateGraph(State)
builder.add_node("charter",charter)
builder.add_node("rule",rule)
builder.add_node("place",place)
builder.add_node("event",event)
builder.add_node("item",item)
builder.add_node("group",group)

builder.set_entry_point("charter")
builder.add_edge("charter","rule")
builder.add_edge("rule","place")
builder.add_edge("place","event")
builder.add_edge("event","item")
builder.add_edge("item","group")
builder.add_edge("group",END)

detail = builder.compile()


### 설정 비교

## FastAPI

In [30]:
app = FastAPI()
users : Dict[str, State] = {}

# 모든출처(origin)에서의 접근을허용하기위한설정
origins = ["https://ai0917-kt-aivle-shool-8th-bigprojec-eta.vercel.app"]
# CORS(Cross-Origin Resource Sharing) 미들웨어 추가
# 모든도메인, 쿠키,인증정보, HTTP 메소드, HTTP 헤더허용
app.add_middleware(
CORSMiddleware,
allow_origins=origins,
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],  
allow_headers=origins,
)
@app.get("/")
def connect_check():
    return "AI 서버와 연결되었습니다."

@app.post("/Categories")
async def process_cat(data : dict = Body(...)):
    global current_state
    initial_state : State = {
        "title" : data.get("title",""),
        "txt" : data.get("txt",""),
        "charter":{},
        "check":dict(),
        "episode": data.get("episode",""),
        "subtitle": data.get("subtitle",""),
        "userid" : data.get("userid",""),
        "캐릭터" : dict(),
        "세계관" : dict(),
        "장소" : dict(),
        "사건" : dict(),
        "아이템" : dict(),
        "단체" : dict()
    }
    userid = initial_state.get('userid')
    result = cat.invoke(initial_state)
    users[userid] = result
    return {
        "check" : result.get("check",{})
    }

@app.post("/choice")
async def process_choice(data : dict = Body(...)):
    userid = data.get("userid")
    categories = data.get("check", [])

    if userid not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
        
    user_state = users[userid]  #기존의 데이/터 가져오기
    user_state["check"] = categories
    user_state = detail.invoke(user_state)
    user_state['txt'] = ""
    return user_state

@app.post("/comparison")
async def process_comparison(data : dict = Body(...)):
    userid = data.get("userid")
    if userid not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    users[userid] = data
    user_state = data
    result = comparison(user_state)
    return result

@app.post("/renew")
async def process_renew(data : dict = Body(...)):
    userid = data.get("userid")
    if userid not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    editlis = data.get("설정 결합")
    for i in editlis:
        a = edit(i[0],i[1],i[2])
    newlis = data.get("신규 업로드")
    user_state = users[userid]
    b = upload_vector(user_state,newlis)
    return [a,b]

@app.get

## 테스트

In [102]:
state = {'txt': """오래전 잊힌 왕국 카마보르에는 왕좌와 멀리 떨어진 마을에 사는 사람들이 있었다. 바로 이 시골 마을에서 한 평범한 재봉사가 사랑스러운 인형 그웬을 만들었다.

그웬은 과거를 떠올릴 때 사랑을 느꼈다. 재봉사와 그웬은 종일 뭔가를 만들며 하루를 보냈다. 재봉사는 가만히 있는 그웬의 손 위에 가위를 올려놓고 근처에서 바늘과 실로 바느질을 하곤 했다. 밤이면 둘은 식탁 아래에서 쭈그리고 앉아 결투 아닌 결투를 벌였다. 그럴 때면 촛불로 밝힌 주방에 가위와 식기가 부딪치는 소리가 울렸다.

그러나 곧 놀이가 끝나며 빛이 희미해졌다. 이유는 알 수 없었으나 그웬은 자세한 기억을 떠올리려고 애쓸 때마다 찌릿찌릿한 고통을 느꼈다. 이름과 얼굴이 떠오르지 않는 한 남자와 연관이 있었다. 그웬은 수 세기 동안 그 자리에서 조용히 잊혀 갔다. 그웬의 기억은 시간이 흐르며 바다의 파도와 함께 쓸려 갔다.

그러던 어느 밤 그웬의 눈이 떠졌다. 그웬은 집에서 멀리 떨어진 어두운 바닷가에서 처음으로 깨어났다. 알 수 없는 마법의 힘 덕분에 마음대로 손과 발을 움직일 수 있는 살아 있는 사람이 된 것이었다!

그웬은 삶을 기쁘게 받아들였다. 모래 위를 팔짝팔짝 뛰어다니고 눈으로 먼 곳을 볼 수 있다는 사실에 감탄하며 피부에 느껴지는 조약돌의 감각과 등에 느껴지는 바람에 감동했다. 그때 해안을 따라 아주 오래전 버려진 듯한 잔해가 흩어져 있는 모습이 눈에 들어왔다. 망가진 상자 옆에 묘하게 익숙한 도구가 널브러져 있었다.

가위와 바늘과 실이었다.

그웬은 그 도구를 바로 알아봤다. 자신을 만든 창조자의 도구였다. 그웬의 손가락이 도구에 닿자 손에서 빛으로 반짝거리는 안개가 터지듯 흘러나왔다. 안개는 안전하고 따뜻하게 느껴졌다. 마치 소중한 과거의 품에 안기는 것 같았다.

하지만 이 마법에 이끌린 것은 그웬만이 아니었다.

섬에 도사리고 있던 다른 안개가 몰려들었다. 검은빛 안개는 감기고 뒤틀리며 무시무시한 망령들의 모습으로 변했다. 그웬이 새로 찾은 무엇인가에 이끌린 망령들은 그것을 갈구하며 집착했다.

그웬은 망령들이 달려들어도 흔들리지 않고 가위를 내질렀다. 기쁘게도 점점 더 많은 안개가 대기를 채우며 도구의 크기와 힘에 마법을 걸었다. 그저 강철에 불과했던 도구는 무형의 마법이 되었다.

하지만 망령들은 끈질겼다. 끊임없이 커지는 검은 안개의 힘으로 수가 계속해서 늘어만 갔다. 그웬은 비통하면서도 기묘하게 익숙한 고통을 느끼기 시작했다. 주위를 둘러싼 망령 사이에서 억눌린 기억이 수면 위로 떠올랐다. 그웬을 만든 창조자는 다쳐서 고통스럽게 누워 있었다. 그웬은 창조자 옆에 있던 남자의 얼굴을 마침내 떠올렸다.

'비에고.'

남자의 이름을 기억한 그웬은 무릎을 털썩 꿇었다. 창조자와 함께 보낸 더 행복하고 소박했던 시절을 아련하게 떠올렸다. 그리고 마지막으로 가위를 슬쩍 보았다.8<-8<-8<-8<-

그때 그웬은 놀라운 사실을 깨달았다. 그 남자의 뒤틀린 허영심에 희생된 창조자는 아예 사라진 게 아니었다. 그웬을 처음으로 꿰매어 완성한 창조자의 바로 그 도구가 이제 그웬의 손에 있었다. 이것은 우연이 아니었다. 그웬은 마음속 깊은 곳에서 그녀의 창조자가 아직 자신과 함께 싸우고 있다는 것을 알았다.

이러한 선물을 당연히 여길 수는 없었다.

바늘과 실을 잡은 그웬은 신성한 안개를 휘둘러 몰려드는 망령들을 밀어냈다. 그웬의 가위가 빠르고 세게 공기를 갈랐다. 식탁 밑에서 창조자와 함께 대전투를 상상하던 행복한 밤으로 돌아간 것만 같았다. 얼마 후 망령의 모습은 더 이상 보이지 않았다.

그웬은 승리했지만 이것이 겨우 시작이라는 사실을 깨달았다. 망령들과 비에고는 연결되어 있었다. 두 존재 모두 엄청난 고통을 퍼뜨리는 원인이었다. 지체할 시간이 없었다. 그웬은 검은 안개를 쫓아가 반드시 막겠다고 다짐했다. 결코 쉽지 않은 도전이었지만, 그웬은 살아 있는 매 순간을 즐겼다. 지금의 축복이 언제까지 이어질지 알 수 없었기 때문이다.

두 번 다시 없을 삶을 살게 된 그웬은 온갖 역경에 굳건히 맞서는 선한 힘이 되기로 했다. 룬테라 전역을 다니며 다치고 괴로워하는 이들에게 기쁨을 되돌려 주기로 결심한 것이다. 순간순간이 소중한 그웬은 늘 자신의 목적을 되새기며 발걸음을 내딛는다..""",
 'episode': 3,
 'subtitle': '그웬',
 'check': dict(),
 'title': '리그오브레전드 스토리',
 'userid': 'aaaaaaaa',
 '캐릭터': dict(),
 '세계관': dict(), 
 '장소': dict(),
 '사건': dict(),
 '단체': dict(),
 '아이템' : dict()
        }
state = cat.invoke(state)


목록 추출 완료
{
'캐릭터':['그웬', '재봉사', '비에고', '망령'],
'세계관':['마법', '안개', '기억', '고통', '희생'],
'장소':['카마보르', '시골 마을', '바닷가', '식탁 아래', '룬테라'],
'사건':['그웬의 탄생', '그웬의 각성', '망령과의 전투', '비에고와의 연결', '기억의 회복', '그웬의 여정'],
'아이템':['그웬', '가위', '바늘', '실', '도구', '상자', '조약돌', '촛불'],
'단체':[]
}


In [103]:
state = detail.invoke(state)

{
  "그웬": [
    {
      "별명": "사랑스러운 인형",
      "정체성": {
        "종족": "Unknown",
        "연령": "수 세기",
        "직업/신분": "인형",
        "소속단체/가문": "카마보르의 시골 마을"
      },
      "외형": "Unknown",
      "성격": ["기쁨을 받아들임", "굳건함", "선함"],
      "기술/능력": [
        {
          "기술명": "가위 사용",
          "숙련도/강도": "높음",
          "상세효과": "마법적인 힘을 받아 대기를 가르고 망령을 물리침"
        },
        {
          "기술명": "바늘과 실 사용",
          "숙련도/강도": "높음",
          "상세효과": "신성한 안개를 휘둘러 망령을 밀어냄"
        },
        {
          "기술명": "마법적인 안개 방출",
          "숙련도/강도": "Unknown",
          "상세효과": "손에서 빛으로 반짝이는 안개가 터져 나오며 안전하고 따뜻한 느낌을 줌"
        }
      ],
      "인물관계": [
        {
          "대상이름": "재봉사",
          "관계": "창조자",
          "상세내용": "그웬을 만들고 함께 시간을 보내며 결투 놀이를 함"
        },
        {
          "대상이름": "비에고",
          "관계": "과거의 연관성",
          "상세내용": "그웬의 기억 속에서 창조자와 관련된 남자. 뒤틀린 허영심으로 창조자를 희생시킴"
        }
      ]
    }
  ]
}
변환 성공!
{
  "재봉사": [
    {
      "별명": "Unknown",
      "정체성": {
        "종족":

In [80]:
comparison(state)

[결과: 충돌]
[판단사유: 기존설정과 신규설정은 '그웬'이라는 동일한 대상에 대해 근본적인 정체성, 성격, 인물관계, 존재론적 상태 등 모든 핵심 설정에서 예외 없이 물리적으로 불가능한 수준의 모순을 보이며 충돌합니다.

**주요 충돌 지점:**

1.  **정체성 (종족/직업/신분):**
    *   **기존:** '인형에서 마법의 힘으로 살아있는 사람이 됨', '살아있는 존재, 선한 힘'
    *   **신규:** '인형에서 마법의 힘으로 죽어있는 언데드가 됨', '죽어있는 존재, 악한 존재'
    *   **충돌:** '살아있는 사람'과 '죽어있는 언데드'는 동시에 존재할 수 없으며, '선한 힘'과 '악한 존재'는 본질적으로 상반됩니다.

2.  **외형:**
    *   **기존:** '사랑스러운 인형이었음', '피부와 등에 감각을 느낌'
    *   **신규:** '저주인형이었음', '증오를 느낌'
    *   **충돌:** '사랑스러운'과 '저주받은'은 외형적 묘사에서 상반되며, '감각을 느낌' (살아있는 존재의 특성)과 '증오를 느낌' (언데드의 감정)은 존재론적 상태와 연결되어 충돌합니다.

3.  **성격:**
    *   **기존:** '사랑을 느낌', '기쁨을 받아들임', '선함' 등 긍정적이고 선한 감정 및 특성
    *   **신규:** '증오를 느낌' 등 부정적이고 악한 감정
    *   **충돌:** '선함'과 '증오'는 한 인물의 주된 성격으로 동시에 양립하기 어렵습니다.

4.  **기술/능력:**
    *   **기존:** '빛으로 반짝거리는 안개' (안전하고 따뜻함), '신성한 안개 휘두르기' (망령 밀어냄), '룬테라 전역을 다니며 기쁨 되돌려주기' (선한 목적)
    *   **신규:** 기술/능력 없음.
    *   **충돌:** 신규설정에 기술/능력이 없다는 것은 직접적인 충돌은 아니지만, 기존설정의 기술들이 '선한 힘'과 '기쁨'을 목적으로 하는 반면, 신규설정의 그웬은 '악한 존재'이므로 기존

{'캐릭터': [['그웬',
   "[결과: 충돌]\n[판단사유: 기존설정과 신규설정은 '그웬'이라는 동일한 대상에 대해 근본적인 정체성, 성격, 인물관계, 존재론적 상태 등 모든 핵심 설정에서 예외 없이 물리적으로 불가능한 수준의 모순을 보이며 충돌합니다.\n\n**주요 충돌 지점:**\n\n1.  **정체성 (종족/직업/신분):**\n    *   **기존:** '인형에서 마법의 힘으로 살아있는 사람이 됨', '살아있는 존재, 선한 힘'\n    *   **신규:** '인형에서 마법의 힘으로 죽어있는 언데드가 됨', '죽어있는 존재, 악한 존재'\n    *   **충돌:** '살아있는 사람'과 '죽어있는 언데드'는 동시에 존재할 수 없으며, '선한 힘'과 '악한 존재'는 본질적으로 상반됩니다.\n\n2.  **외형:**\n    *   **기존:** '사랑스러운 인형이었음', '피부와 등에 감각을 느낌'\n    *   **신규:** '저주인형이었음', '증오를 느낌'\n    *   **충돌:** '사랑스러운'과 '저주받은'은 외형적 묘사에서 상반되며, '감각을 느낌' (살아있는 존재의 특성)과 '증오를 느낌' (언데드의 감정)은 존재론적 상태와 연결되어 충돌합니다.\n\n3.  **성격:**\n    *   **기존:** '사랑을 느낌', '기쁨을 받아들임', '선함' 등 긍정적이고 선한 감정 및 특성\n    *   **신규:** '증오를 느낌' 등 부정적이고 악한 감정\n    *   **충돌:** '선함'과 '증오'는 한 인물의 주된 성격으로 동시에 양립하기 어렵습니다.\n\n4.  **기술/능력:**\n    *   **기존:** '빛으로 반짝거리는 안개' (안전하고 따뜻함), '신성한 안개 휘두르기' (망령 밀어냄), '룬테라 전역을 다니며 기쁨 되돌려주기' (선한 목적)\n    *   **신규:** 기술/능력 없음.\n    *   **충돌:** 신규설정에 기술/능력이 없다는 것은 직접적인 충돌은 아니지만, 기존설정의 기술들이 '선한 힘'

### 폐기

In [ ]:
from pprint import pprint
import requests
domain = "https://salad-silent-birth-varying.trycloudflare.com/"
url = domain + "/api/chat"

headers = {"X-API-KEY": "aivle202508"}

data = {
    "model": "gemma2:9b",
    "messages": [
        {
            "role": "system",
            "content": "대답은 한국어로 하세요. 이모지는 빼도록 하세요.",,
        },
        {"role": "user", "content": "server sent events 에 대해 설명해 줘."},,
    ],
    "stream": False,
}

response = requests.post(url, json=data, headers=headers)
pprint(response.text)


In [ ]:
state['check'] = {'캐릭터': ['비에고'],
 '사건': ['형의 갑작스러운 죽음',
  '이졸데와의 만남과 청혼',
  '왕국 붕괴 시작',
  '자객의 습격과 이졸데의 중독',
  '이졸데의 죽음',
  '축복의 물을 이용한 이졸데의 부활 시도',
  '이졸데의 망령화와 비에고 공격',
  '군도 전역의 파멸',
  '비에고의 재기'],
}

In [ ]:
my_dict = {"character_name": "비에고"}

# 키(Key) 가져오기
key = next(iter(my_dict))  # "character_name"

# 값(Value) 가져오기
value = next(iter(my_dict.values()))  # "비에고"
print(my_dict,value,key)

In [76]:
state= {'txt': '바다 건너 동쪽 먼 곳의 왕국, 해안을 따라 점점이 남은 폐허들에서나 간간이 들리는 이름을 아는 이는 적다. 그 왕국의 젊고 어리석은 군주, 사랑에 빠져 왕국을 파멸시킬 운명을 타고난 왕자를 아는 이는 더더욱 적다.\n\n그 이름은 비에고, 세상 모두에게 중대한 위협이 된 남자다.\n\n선왕의 둘째 아들로 태어난 비에고는 왕좌를 물려받을 후계자가 아니었다. 그 대신 안락한 삶을 영위하며 자만심과 이기심만 키웠으나, 형의 갑작스러운 죽음에 그는 왕국을 다스릴 적성도, 바람도 없이 얼떨결에 왕관을 쓰게 되었다.\n\n가난한 재봉사 이졸데를 만나기 전까지, 비에고는 왕좌에 관심이 없었다. 이졸데의 아름다움에 사로잡힌 젊은 왕 비에고는 그녀에게 청혼했고, 그렇게 세계에서 손꼽히는 권력을 가진 왕이 시골 소녀와 맺어졌다.\n\n두 사람의 연애는 매혹적이었다. 타인에게 거의 관심을 보이지 않던 비에고는 자신의 삶을 이졸데에게 바쳤다. 둘은 언제나 함께였다. 비에고는 이졸데를 두고 나다니는 일이 드물었고, 자신의 왕비에게 선물을 아끼지 않았으며, 이졸데가 곁에 있을 때는 다른 곳을 쳐다보지 않았다.\n\n비에고의 신하들은 성이 났다. 어떻게 해도 비에고가 정사를 돌보지 않고, 미심쩍은 통치 아래 왕국이 붕괴하기 시작하자, 몇몇이 은밀히 모여 나랏일에 관심이 없는 새 왕을 끌어내리고자 모의했다. 한편, 왕국의 적들은 이를 공격의 기회로 보았다. 독사들이 꽈리를 틀기 시작했다.\n\n어느 날, 독 묻은 단검을 지닌 자객이 비에고를 찾아왔다. 하지만 왕을 둘러싼 삼엄한 경비 탓에 단검은 표적을 찌르지 못하고 이졸데를 스쳤다.\n\n독이 빠르게 퍼져서 이졸데는 극심한 무기력증에 빠졌다. 비에고는 아내의 상태가 점점 위중해지는 걸 지켜볼 수밖에 없었다. 분노와 절망에 압도당한 비에고는 아내를 구하고자 나라의 모든 재물을 탕진했다.\n\n하지만 모두 헛된 일이었다. 이졸데는 침대에서 죽어갔고, 광기가 비에고를 집어삼켰다.\n\n비에고는 더 맹렬하고 필사적으로 해독제를 찾았다. 아내의 죽음을 받아들일 수 없었던 비에고는 그녀를 되찾고자 국고를 모조리 털어 동전 하나 남기지 않았다. 이내 왕국이 혼란에 빠지자, 증오와 분노로 가득한 비에고는 죽은 이졸데와 칩거에 들어갔다.\n\n어느 날 비에고는 축복의 빛 군도에 어떤 병이든 치유하는 물이 있다는 비밀을 알게 되었다. 그는 강력한 군세를 이끌고 평화로운 군도에 쳐들어가, 길을 가로막는 이들을 모조리 학살하고 심부의 성소에 도달해 아내의 몸을 축복받은 물속에 담갔다. 이졸데가 돌아올 수만 있다면 그가 불러온 파멸 따윈 상관없었다. 치르지 못할 대가는 없었다.\n\n잠시나마, 이졸데가 정말로 돌아왔다.\n\n이졸데는 그림자와 격노의 끔찍한 망령으로 되살아났다. 그녀는 죽음의 품에서 강제로 떨어진 충격, 분노, 고통에 비에고의 마법 검을 들어 그를 찔렀다. 축복받은 물에 담긴 마법과 고대의 검이 격돌하자, 방의 기운이 폭발하며 군도 전역을 찢어놓고 그에 닿은 이들 모두를 의식을 유지한 채 고통받는 언데드로 만들었다.\n\n그런데도 비에고는 아무것도 기억하지 못한다. 자신의 나라가 무너져 폐허가 되었고, 위대한 국가들이 세워지고 멸망했으며, 그의 이름마저 잊혔으나... 죽음 이후 천년이 지난 지금, 비에고가 다시 일어섰다. 이번에는 실패하지 않으리라.\n\n생에서 품었던 것과 같은 위험한 집착이 비에고의 정신을 일그러트린다. 괴기하며 흔들리지 않는 사랑이 그의 모든 행동과 소망과 잔악함을 부추긴다. 상처받은 심장에서 치명적인 검은 안개가 쏟아져 지나는 곳마다 생명을 앗아간다. 검은 안개로 세상을 뒤져 이졸데를 되찾을 방법을 찾아낼 것이다.\n\n비에고 앞에 쓰러진 군단들이 그의 종으로 다시 일어날 것이며, 살아있는 어둠이 대륙들을 집어삼키리라. 세상은 사랑의 열병을 앓던 고대 군주의 행복을 앗아간 대가를 남김없이 치를 것이다. 이졸데의 얼굴을 다시 볼 수만 있다면, 그는 어떤 파괴를 불러오든 눈 하나 깜짝하지 않는다.\n\n비에고의 통치는 공포다.\n\n비에고의 사랑은 영원하다.\n\n이졸데를 되찾을 때까지, 누구도 몰락한 왕의 길을 막지 못할 것이다.',
 'episode': 1,
 'subtitle': '그림자군도',
 'check': {'캐릭터': ['비에고', '이졸데'],
  '세계관': ['왕국이 파멸할 운명',
   '독 묻은 단검',
   '축복의 빛 군도',
   '병을 치유하는 물',
   '그림자와 격노의 망령',
   '의식을 유지한 채 고통받는 언데드',
   '천년 후 다시 일어섬',
   '검은 안개',
   '살아있는 어둠'],
  '장소': ['바다 건너 동쪽 먼 곳의 왕국', '해안', '축복의 빛 군도', '심부의 성소'],
  '사건': ['형의 갑작스러운 죽음',
   '이졸데와의 만남과 청혼',
   '왕국 붕괴 시작',
   '자객의 습격과 이졸데의 중독',
   '이졸데의 죽음',
   '축복의 물을 이용한 이졸데의 부활 시도',
   '이졸데의 망령화와 비에고 공격',
   '군도 전역의 파멸',
   '비에고의 재기'],
  '아이템': ['독 묻은 단검', '마법 검'],
  '단체': ['비에고의 신하들', '왕국의 적들']},
 'title': '리그오브레전드 스토리',
 'userid': 'aaaaaaaa',
 '캐릭터': {'비에고': [{'별명': '세상 모두에게 중대한 위협이 된 남자',
    '정체성': {'종족': 'Unknown',
     '연령': '젊은',
     '직업/신분': '왕자, 왕',
     '소속단체/가문': '동쪽 먼 곳의 왕국'},
    '외형': 'Unknown',
    '성격': ['자만심',
     '이기심',
     '관심 없음 (이졸데를 만나기 전)',
     '매혹됨 (이졸데에게)',
     '헌신적 (이졸데에게)',
     '분노',
     '절망',
     '광기',
     '맹렬함',
     '필사적',
     '증오',
     '위험한 집착',
     '괴기함',
     '흔들리지 않는 사랑',
     '잔악함'],
    '기술/능력': [{'기술명': '마법 검',
      '숙련도/강도': 'Unknown',
      '상세효과': '이졸데를 찔렀을 때 축복받은 물과 격돌하여 군도 전역을 찢고 의식을 유지한 채 고통받는 언데드로 만듦. 상처받은 심장에서 치명적인 검은 안개를 쏟아내며 지나는 곳마다 생명을 앗아감.'},
     {'기술명': '검은 안개',
      '숙련도/강도': '치명적인',
      '상세효과': '상처받은 심장에서 쏟아져 나와 지나는 곳마다 생명을 앗아감. 이졸데를 되찾을 방법을 찾아냄.'}],
    '인물관계': [{'대상이름': '이졸데',
      '관계': '연인, 왕비',
      '상세내용': '가난한 재봉사였으나 비에고의 사랑을 받음. 비에고는 그녀의 아름다움에 사로잡혀 청혼했고, 그녀에게 자신의 삶을 바쳤으며, 그녀에게 선물을 아끼지 않았고, 그녀가 곁에 있을 때는 다른 곳을 보지 않았다. 그녀의 죽음으로 인해 광기에 휩싸여 해독제를 필사적으로 찾았고, 그녀를 되찾기 위해 국고를 탕진했으며, 그녀를 되살리기 위해 축복의 빛 군도에 쳐들어가 학살을 벌였다. 그녀는 그림자와 격노의 망령으로 되살아났고, 비에고의 마법 검으로 그를 찔렀다. 비에고는 이졸데를 되찾기 위해서라면 어떤 파괴도 불사한다.'},
     {'대상이름': '선왕', '관계': '아버지', '상세내용': '선왕의 둘째 아들로 태어났다.'},
     {'대상이름': '형', '관계': '형', '상세내용': '형의 갑작스러운 죽음으로 왕좌를 물려받게 되었다.'},
     {'대상이름': '신하들',
      '관계': '신하',
      '상세내용': '비에고가 정사를 돌보지 않자 나랏일에 관심 없는 왕을 끌어내리고자 모의했다.'},
     {'대상이름': '왕국의 적들',
      '관계': '적',
      '상세내용': '비에고의 미심쩍은 통치 아래 왕국이 붕괴하기 시작하자 공격의 기회로 보았다.'}]}],
  '이졸데': [{'별명': 'Unknown',
    '정체성': {'종족': 'Unknown',
     '연령': 'Unknown',
     '직업/신분': '가난한 재봉사',
     '소속단체/가문': 'Unknown'},
    '외형': '아름다움',
    '성격': ['Unknown'],
    '기술/능력': [{'기술명': '그림자와 격노의 끔찍한 망령',
      '숙련도/강도': 'Unknown',
      '상세효과': '죽음의 품에서 강제로 떨어진 충격, 분노, 고통으로 인해 비에고의 마법 검을 들어 그를 찔렀다. 축복받은 물에 담긴 마법과 고대의 검이 격돌하자, 방의 기운이 폭발하며 군도 전역을 찢어놓고 그에 닿은 이들 모두를 의식을 유지한 채 고통받는 언데드로 만들었다.'}],
    '인물관계': [{'대상이름': '비에고',
      '관계': '연인/왕비',
      '상세내용': '비에고는 이졸데의 아름다움에 사로잡혀 청혼했고, 세계에서 손꼽히는 권력을 가진 왕이 시골 소녀와 맺어졌다. 비에고는 자신의 삶을 이졸데에게 바쳤으며, 둘은 언제나 함께였다. 비에고는 이졸데를 두고 나다니는 일이 드물었고, 자신의 왕비에게 선물을 아끼지 않았으며, 이졸데가 곁에 있을 때는 다른 곳을 쳐다보지 않았다. 이졸데가 독에 중독되어 죽어갈 때, 비에고는 아내를 구하고자 나라의 모든 재물을 탕진했다. 이졸데의 죽음 이후 비에고는 광기에 휩싸여 그녀를 되찾고자 국고를 모조리 털었다. 비에고는 축복의 빛 군도에서 이졸데를 되살렸으나, 그녀는 망령으로 되살아나 비에고를 찔렀다. 비에고는 이졸데를 되찾기 위해 어떤 파괴도 불사한다.'}]}]},
 '세계관': {'왕국이 파멸할 운명': '젊고 어리석은 군주 비에고는 사랑에 빠져 왕국을 파멸시킬 운명을 타고났으며, 그의 광적인 집착과 파괴적인 행동은 결국 왕국의 몰락을 초래했다.',
  '독 묻은 단검': '비에고 왕을 암살하려던 자객이 휘두른 단검으로, 비에고 대신 왕비 이졸데를 스쳐 치명적인 독을 퍼뜨렸다.',
  '축복의 빛 군도': '모든 병을 치유하는 물이 있는 곳으로, 비에고가 이졸데를 되살리기 위해 침략하여 마법 검과 축복받은 물의 충돌로 인해 군도 전역이 찢어지고 닿은 이들이 고통받는 언데드가 되었다.',
  '병을 치유하는 물': '축복의 빛 군도에 있는 이 물은 어떤 병이든 치유하는 힘을 가지고 있습니다.',
  '그림자와 격노의 망령': '이졸데는 죽음에서 강제로 돌아온 충격, 분노, 고통으로 인해 비에고의 마법 검으로 그를 찔렀고, 이로 인해 축복받은 물과 고대의 검이 격돌하며 폭발을 일으켜 군도 전역을 찢고 닿은 모든 이를 의식을 유지한 채 고통받는 언데드로 만들었다.',
  '의식을 유지한 채 고통받는 언데드': '축복받은 물과 고대의 검이 격돌하며 발생한 폭발로 인해, 닿은 모든 존재는 의식을 유지한 채 고통받는 언데드가 되었다.',
  '천년 후 다시 일어섬': '사랑하는 이를 되찾기 위해 죽음 이후 천 년 만에 다시 깨어난 비에고는, 그의 길을 막는 모든 것을 파멸시키며 검은 안개를 퍼뜨려 세상을 집어삼킬 것이다.',
  '검은 안개': '비에고의 상처받은 심장에서 뿜어져 나오는 치명적인 검은 안개는 닿는 모든 생명을 앗아가며, 비에고는 이 안개를 이용해 죽은 연인 이졸데를 되찾으려 한다.',
  '살아있는 어둠': '비에고의 슬픔과 집착에서 비롯된 치명적인 검은 안개로, 닿는 모든 생명을 앗아가고 언데드로 만들어 세상을 집어삼키는 파멸적인 힘이다.'},
 '장소': {'바다 건너 동쪽 먼 곳의 왕국': {'별칭': 'Unknown',
   '지역정체성': {'분위기': '폐허',
    '역사': '간간이 들리는 이름을 아는 이는 적다. 왕국의 젊고 어리석은 군주, 사랑에 빠져 왕국을 파멸시킬 운명을 타고난 왕자. 선왕의 둘째 아들로 태어난 비에고가 왕좌를 물려받았으나, 왕국을 다스릴 적성도, 바람도 없이 얼떨결에 왕관을 쓰게 되었다. 가난한 재봉사 이졸데를 만나 사랑에 빠졌고, 비에고의 신하들은 성이 났다. 미심쩍은 통치 아래 왕국이 붕괴하기 시작했다. 왕국의 적들은 공격의 기회로 보았다. 비에고는 아내 이졸데를 구하고자 나라의 모든 재물을 탕진했으나 헛된 일이었다. 이졸데는 침대에서 죽어갔고, 광기가 비에고를 집어삼켰다. 비에고는 더 맹렬하고 필사적으로 해독제를 찾았다. 국고를 모조리 털어 동전 하나 남기지 않았다. 이내 왕국이 혼란에 빠지자, 증오와 분노로 가득한 비에고는 죽은 이졸데와 칩거에 들어갔다. 축복의 빛 군도에 어떤 병이든 치유하는 물이 있다는 비밀을 알게 되었다. 강력한 군세를 이끌고 평화로운 군도에 쳐들어가, 길을 가로막는 이들을 모조리 학살하고 심부의 성소에 도달해 아내의 몸을 축복받은 물속에 담갔다. 이졸데가 돌아올 수만 있다면 그가 불러온 파멸 따윈 상관없었다. 치르지 못할 대가는 없었다. 이졸데는 그림자와 격노의 끔찍한 망령으로 되살아났다. 그녀는 죽음의 품에서 강제로 떨어진 충격, 분노, 고통에 비에고의 마법 검을 들어 그를 찔렀다. 축복받은 물에 담긴 마법과 고대의 검이 격돌하자, 방의 기운이 폭발하며 군도 전역을 찢어놓고 그에 닿은 이들 모두를 의식을 유지한 채 고통받는 언데드로 만들었다. 죽음 이후 천년이 지난 지금, 비에고가 다시 일어섰다. 이번에는 실패하지 않으리라. 생에서 품었던 것과 같은 위험한 집착이 비에고의 정신을 일그러트린다. 괴기하며 흔들리지 않는 사랑이 그의 모든 행동과 소망과 잔악함을 부추긴다. 상처받은 심장에서 치명적인 검은 안개가 쏟아져 지나는 곳마다 생명을 앗아간다. 검은 안개로 세상을 뒤져 이졸데를 되찾을 방법을 찾아낼 것이다. 비에고 앞에 쓰러진 군단들이 그의 종으로 다시 일어날 것이며, 살아있는 어둠이 대륙들을 집어삼키리라. 세상은 사랑의 열병을 앓던 고대 군주의 행복을 앗아간 대가를 남김없이 치를 것이다. 이졸데의 얼굴을 다시 볼 수만 있다면, 그는 어떤 파괴를 불러오든 눈 하나 깜짝하지 않는다. 비에고의 통치는 공포다. 비에고의 사랑은 영원하다. 이졸데를 되찾을 때까지, 누구도 몰락한 왕의 길을 막지 못할 것이다.',
    '위치': '바다 건너 동쪽 먼 곳'},
   '단체': ['비에고의 왕국'],
   '하부지역': ['축복의 빛 군도']},
  '해안': {'별칭': 'Unknown',
   '지역정체성': {'분위기': '폐허가 간간이 남은 곳', '역사': 'Unknown', '위치': '동쪽 먼 곳의 왕국'},
   '단체': ['Unknown'],
   '하부지역': ['Unknown']},
  '축복의 빛 군도': {'별칭': 'Unknown',
   '지역정체성': {'분위기': '평화로운',
    '역사': '비에고가 이졸데를 되살리기 위해 쳐들어와 축복받은 물에 이졸데의 몸을 담갔을 때, 축복받은 물의 마법과 고대의 검이 격돌하며 군도 전역을 찢어놓고 그에 닿은 이들 모두를 의식을 유지한 채 고통받는 언데드로 만들었다.',
    '위치': '바다 건너 동쪽 먼 곳의 왕국에서 멀리 떨어진 곳'},
   '단체': ['Unknown'],
   '하부지역': ['심부의 성소']},
  '심부의 성소': {'별칭': 'Unknown',
   '지역정체성': {'분위기': '축복받은 물이 있는 곳, 이졸데가 되살아난 장소, 마법과 고대의 검이 격돌하여 폭발이 일어난 장소',
    '역사': '비에고가 이졸데를 되살리기 위해 축복받은 물을 사용한 장소. 이 과정에서 폭발이 일어나 군도 전역을 찢어놓고 닿은 이들을 고통받는 언데드로 만들었다.',
    '위치': '축복의 빛 군도'},
   '단체': ['Unknown'],
   '하부지역': ['Unknown']}},
 '사건': {'형의 갑작스러운 죽음': '비에고의 형이 갑작스럽게 죽으면서, 왕좌에 대한 관심이 없던 비에고가 얼떨결에 왕위를 계승하게 되었다.',
  '이졸데와의 만남과 청혼': '왕 비에고가 가난한 재봉사 이졸데의 아름다움에 사로잡혀 왕좌에 관심이 없다가 그녀에게 청혼하며 세계에서 손꼽히는 권력을 가진 왕이 시골 소녀와 맺어졌다. 이 사건은 비에고의 통치에 대한 무관심과 왕국의 붕괴를 초래하는 원인이 되었다.',
  '왕국 붕괴 시작': '젊고 어리석은 왕 비에고가 사랑에 빠져 정사를 돌보지 않자 왕국이 쇠퇴하고, 왕비 이졸데가 독에 중독되어 죽어가자 왕은 그녀를 살리기 위해 모든 것을 탕진하고 결국 왕국을 파멸로 이끌었다.',
  '자객의 습격과 이졸데의 중독': '비에고의 통치에 불만을 품은 자객이 왕을 노렸으나 빗나가 왕비 이졸데를 독단검으로 찔러 중독시켰고, 이는 비에고의 광기와 왕국 몰락의 시작이 되었다.',
  '이졸데의 죽음': '비에고 왕을 향한 암살자의 독 묻은 단검이 빗나가 왕비 이졸데를 찔러 사망하게 되었고, 이에 분노한 비에고는 이졸데를 되살리기 위해 모든 것을 파멸시킨다.',
  '축복의 물을 이용한 이졸데의 부활 시도': '비에고 왕이 죽은 왕비 이졸데를 되살리기 위해 축복의 물을 사용했으나, 오히려 이졸데를 끔찍한 망령으로 만들고 군도에 재앙을 불러왔다.',
  '이졸데의 망령화와 비에고 공격': '비에고의 왕비 이졸데가 독에 중독되어 죽자, 비에고는 그녀를 되살리기 위해 축복받은 물을 사용했으나, 이졸데는 그림자와 격노의 망령으로 되살아나 비에고의 마법 검으로 그를 찔렀고, 이로 인해 군도 전체가 찢어지며 비에고의 군대가 언데드로 변모했다.',
  '군도 전역의 파멸': '비에고 왕이 사랑하는 이졸데를 되살리기 위해 축복받은 물을 사용했으나, 그 과정에서 발생한 마법 폭발로 군도 전역이 파멸하고 언데드가 창궐하게 되었다.',
  '비에고의 재기': '사랑하는 이졸데를 되살리기 위해 금지된 힘을 사용했다가 파멸을 초래했지만, 천년 후 다시 깨어나 그녀를 되찾기 위해 세상을 파괴하려 한다.'},
 '아이템': {'독 묻은 단검': {'설명': '이졸데에게 치명적인 독을 퍼뜨려 죽음에 이르게 한 단검',
   '관련인물': ['비에고', '이졸데']},
  '마법 검': {'설명': '축복받은 물의 마법과 고대의 검이 격돌하여 폭발을 일으켰으며, 이 폭발로 인해 군도 전역이 찢어졌고 닿은 이들 모두 의식을 유지한 채 고통받는 언데드가 되었다. 비에고의 마법 검은 그의 광기와 집착을 담고 있으며, 검은 안개를 뿜어내 생명을 앗아간다.',
   '관련인물': ['비에고', '이졸데']}},
 '단체': {'비에고의 신하들': {'설명': '비에고의 신하들은 왕이 정사를 돌보지 않고 왕국이 붕괴하기 시작하자, 새 왕을 끌어내리고자 은밀히 모의했다. 또한 비에고 앞에 쓰러진 군단들은 그의 종으로 다시 일어날 것이다.',
   '구성원': ['Unknown']},
  '왕국의 적들': {'설명': '비에고는 선왕의 둘째 아들로 태어나 형의 갑작스러운 죽음으로 왕위를 물려받았으나, 가난한 재봉사 이졸데를 만나 사랑에 빠지면서 왕국을 소홀히 했다. 이졸데가 독 묻은 단검에 의해 죽어가자 비에고는 그녀를 살리기 위해 모든 재물을 탕진했지만 실패했고, 결국 이졸데는 죽었다. 비에고는 이졸데를 되살리기 위해 축복의 빛 군도에 쳐들어가 성소에서 이졸데를 되살렸으나, 그녀는 그림자와 격노의 망령으로 되살아나 비에고를 찔렀다. 이 사건으로 군도 전역이 찢어졌고, 비에고는 천년 후 다시 일어섰다. 그는 이졸데를 되찾기 위해 검은 안개를 퍼뜨리고 세상을 파괴하며, 그의 통치는 공포와 영원한 사랑으로 특징지어진다.',
   '구성원': ['비에고(왕)', '이졸데(왕비)']}}}

In [48]:
response = client.models.generate_content(
    model="gemma-3-27b-it", #gemma-3-12b, gemma-3-1b ,gemma-3-2b, gemma-3-4b
    contents="웹 소설 작가들을 대상으로 공모전을 진행한다고 했을 때, AI를 활용할만한 방법은?"
)


In [49]:
print( response.text)

웹 소설 작가들을 대상으로 하는 공모전을 AI를 활용하여 더욱 풍성하고 효율적으로 만들 수 있는 다양한 방법들이 있습니다. 다음은 몇 가지 아이디어들을 구체적인 예시와 함께 제시합니다.

**1. 공모전 기획 및 홍보 단계**

*   **AI 기반 트렌드 분석:**
    *   최근 웹 소설 시장의 인기 장르, 키워드, 소재 등을 AI가 분석하여 공모전 주제 선정에 활용합니다. (예: "최근 로맨스 판타지에서 '계약 결혼' 소재가 인기 상승", "뱀파이어 소재는 포화 상태")
    *   타겟 독자층의 선호도를 AI가 분석하여 공모전 홍보 문구 및 채널 선정에 반영합니다. (예: "20대 여성 독자층은 인스타그램, 30대 남성 독자층은 웹툰 플랫폼 선호")
*   **AI 기반 홍보 콘텐츠 제작:**
    *   공모전 포스터, 배너, 홍보 영상 등의 초안을 AI가 생성하여 디자이너의 작업 효율성을 높입니다.
    *   공모전 소개 글, SNS 홍보 문구 등을 AI가 다양한 스타일로 작성하여 홍보 효과를 극대화합니다. (예: "흥미진진한 스토리텔링을 기다리는 당신을 위한 기회!", "나만의 판타지 세계를 펼쳐보세요!")
*   **AI 챗봇 활용:**
    *   공모전 관련 문의에 24시간 응대하는 AI 챗봇을 운영하여 참가자 편의성을 높입니다. (예: "공모전 참가 자격은 어떻게 되나요?", "작품 분량 제한은 어떻게 되나요?")

**2. 작품 심사 단계**

*   **AI 기반 표절 검사:**
    *   AI 기반 표절 검사 시스템을 활용하여 표절 의심 작품을 1차적으로 걸러냅니다. (예: CopyKiller, Turnitin 등)
    *   단순 텍스트 일치뿐만 아니라 아이디어, 스토리 구조 등 유사성까지 검사하는 AI 기술을 활용합니다.
*   **AI 기반 작품 분석:**
    *   AI가 작품의 문체, 어휘, 문장 구조, 감정 분석 등을 수행하여 객관적인 평가 지표를 제공합니다. (예: "문체는 간결하고 명확하며, 감정 